# Webcam to HDMI example

Import libraries:

In [1]:
from time import sleep
from pynq.drivers.video import Frame, HDMI
from IPython.display import Image
import cv2
import time
import numpy as np

#### Monitor and camera configurations:

In [2]:
# monitor configuration
hdmi_out_res_mode = 1 # 640x480 @ 60Hz

# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080

# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

#### Initialize camera inout:

In [3]:
# initialize camera from OpenCV
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))

capture device is open: True


#### Initialize and start HDMI output:

In [4]:
# create HDMI output object
hdmi_out=HDMI('out')

# create frame 
hdmi_out.mode(4)
frameOut = hdmi_out.frame(0)
# uncomment the following line to see frame resolution
#print("frame resolution: "str(frameOut.width) +"x" +str(frameOut.height))

# start
resolution = hdmi_out.mode(hdmi_out_res_mode)
hdmi_out.frame_index(0)
hdmi_out.start()

# print configuration
print("selected video resolution: " + resolution)
state = hdmi_out.state()
print("output state is " + str(state))
print("  0: disconnected")
print("  1: streaming")
print("  2: paused")

selected video resolution: 800x600@60Hz
output state is 1
  0: disconnected
  1: streaming
  2: paused


##### Prepare 

In [5]:
# initialize support variables
readError = 0
# check input resolution
ret, frame = videoIn.read()
frame_w = frame.shape[1]
frame_h = frame.shape[0]

#### Stream mode:

In [6]:
start = time.time()
for NumOfFrames in range (100):
            
    # read next image
    ret, frame = videoIn.read()
    frame.resize(frame_w*frame_h*3)
    
    if (ret):
        for y in range (frame_h):
            # in this loop we go through all the lines of the READ image
            # every line has a size of the pixel width multiplied by the number of colors for RGB (3)
            # uncomment the foloowing line for addressing output
            #print("input image width:   " + str(frame_w))
            #print("input image height:  " + str(frame_h))
            #print("output buffer width: " + str(frame_out_w))
            #print("output buffer height:" + str(frame_out_h))
            #print("line copies:")
            #print(str(y*frame_out_w*3)+":"+str(y*frame_out_w*3+frame_w*3)+"="+str(y*frame_w*3)+":"+str((y+1)*frame_w*3))
            #print(str((y*frame_out_w*3+frame_w*3)-y*frame_out_w*3)+"="+str(((y+1)*frame_w*3)-y*frame_w*3))
            frameOut.frame[y*frame_out_w*3:y*frame_out_w*3+frame_w*3] = frame.data[y*frame_w*3:(y+1)*frame_w*3]

        # copy to frame buffer / show on monitor
        hdmi_out.frame(0, frameOut)
        
    else:
        readError = readError + 1

end = time.time()
print("Frames per second:     " + str(100 / (end - start)))
print("Number of read errors: " + str(readError))

Frames per second:     8.1908051343057
Number of read errors: 0


#### Laplacian


In [7]:
start = time.time()
for NumOfFrames in range (100):
            
    # read next image
    ret, frame = videoIn.read()
    
    if (ret):
        #frame.resize(frame_w*frame_h*3)
        frame = cv2.Laplacian(frame, cv2.CV_8U) 
        frame.resize(frame_w*frame_h*3)
        
        for y in range (frame_h):
            # in this loop we go through all the lines of the READ image
            # every line has a size of the pixel width multiplied by the number of colors for RGB (3)
            # uncomment the foloowing line for addressing output
            #print("input image width:   " + str(frame_w))
            #print("input image height:  " + str(frame_h))
            #print("output buffer width: " + str(frame_out_w))
            #print("output buffer height:" + str(frame_out_h))
            #print("line copies:")
            #print(str(y*frame_out_w*3)+":"+str(y*frame_out_w*3+frame_w*3)+"="+str(y*frame_w*3)+":"+str((y+1)*frame_w*3))
            #print(str((y*frame_out_w*3+frame_w*3)-y*frame_out_w*3)+"="+str(((y+1)*frame_w*3)-y*frame_w*3))
            frameOut.frame[y*frame_out_w*3:y*frame_out_w*3+frame_w*3] = frame.data[y*frame_w*3:(y+1)*frame_w*3]

        # copy to frame buffer / show on monitor
        hdmi_out.frame(0, frameOut)
        
    else:
        readError = readError + 1

end = time.time()
print("Frames per second:     " + str(100 / (end - start)))
print("Number of read errors: " + str(readError))

Frames per second:     4.943327048157233
Number of read errors: 0


#### Canny Edge Detection
Canny Edge Detection is a popular edge detection algorithm. It was developed by John F. Canny in 1986. It is a multi-stage algorithm and we will go through each stages.
Doc source: http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html

In [8]:
start = time.time()
for NumOfFrames in range (20):
            
    # read next image
    ret, frame = videoIn.read()
    frame.resize(frame_w*frame_h*3)
    
    if (ret):
        # "Any edges with intensity gradient more than maxVal are sure to be edges and those below 
        # minVal are sure to be non-edges, so discarded. Those who lie between these two thresholds 
        # are classified edges or non-edges based on their connectivity. If they are connected to 
        # “sure-edge” pixels, they are considered to be part of edges. Otherwise, they are also 
        # discarded." 
        frame = cv2.Canny(frame,100,110)
        
        for y in range (frame_h):
            # in this loop we go through all the lines of the READ image
            # every line has a size of the pixel width multiplied by the number of colors for RGB (3)
            # uncomment the foloowing line for addressing output
            #print("input image width:   " + str(frame_w))
            #print("input image height:  " + str(frame_h))
            #print("output buffer width: " + str(frame_out_w))
            #print("output buffer height:" + str(frame_out_h))
            #print("line copies:")
            #print(str(y*frame_out_w*3)+":"+str(y*frame_out_w*3+frame_w*3)+"="+str(y*frame_w*3)+":"+str((y+1)*frame_w*3))
            #print(str((y*frame_out_w*3+frame_w*3)-y*frame_out_w*3)+"="+str(((y+1)*frame_w*3)-y*frame_w*3))
            frameOut.frame[y*frame_out_w*3:y*frame_out_w*3+frame_w*3] = frame.data[y*frame_w*3:(y+1)*frame_w*3]

        # copy to frame buffer / show on monitor
        hdmi_out.frame(0, frameOut)
        
    else:
        readError = readError + 1

end = time.time()
print("Frames per second:     " + str(20 / (end - start)))
print("Number of read errors: " + str(readError))

Frames per second:     0.5371525277489161
Number of read errors: 0


#### SURF (Speeded-Up Robust Features)
opencv_contrib needs to be compiled

start = time.time()
for NumOfFrames in range (20):
            
    # read next image
    ret, frame = videoIn.read()
    frame.resize(frame_w*frame_h*3)
    
    if (ret):        
        # Create SURF object. You can specify params here or later.
        # Here I set Hessian Threshold to 400
        surf = cv2.SURF(400)
        # Find keypoints and descriptors directly
        kp, des = surf.detectAndCompute(frame,None)
        frame = cv2.drawKeypoints(frame,kp,None,(255,0,0),4)
        
        for y in range (frame_h):
            # in this loop we go through all the lines of the READ image
            # every line has a size of the pixel width multiplied by the number of colors for RGB (3)
            # uncomment the foloowing line for addressing output
            #print("input image width:   " + str(frame_w))
            #print("input image height:  " + str(frame_h))
            #print("output buffer width: " + str(frame_out_w))
            #print("output buffer height:" + str(frame_out_h))
            #print("line copies:")
            #print(str(y*frame_out_w*3)+":"+str(y*frame_out_w*3+frame_w*3)+"="+str(y*frame_w*3)+":"+str((y+1)*frame_w*3))
            #print(str((y*frame_out_w*3+frame_w*3)-y*frame_out_w*3)+"="+str(((y+1)*frame_w*3)-y*frame_w*3))
            frameOut.frame[y*frame_out_w*3:y*frame_out_w*3+frame_w*3] = frame.data[y*frame_w*3:(y+1)*frame_w*3]

        # copy to frame buffer / show on monitor
        hdmi_out.frame(0, frameOut)
        
    else:
        readError = readError + 1

end = time.time()
print("Frames per second:     " + str(20 / (end - start)))
print("Number of read errors: " + str(readError))

In [ ]:
# release camera and monitor
videoIn.release()
hdmi_out.stop()